In [1]:
import os
import sys
import pandas as pd
import numpy as np
from copy import copy
import warnings
import importlib
from IPython.core.interactiveshell import InteractiveShell
# from matplotlib import rcParams

# rcParams.update({
#     "text.usetex": True,                
#     "font.family": "serif",            
#     "font.serif": ["Computer Modern"], 
#     "axes.labelsize": 20,              
#     "xtick.labelsize": 20,             
#     "ytick.labelsize": 20,             
#     "legend.fontsize": 20,            
#     "figure.titlesize": 0,             
#     "axes.titlesize": 0,              
# })
notebook_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
    
utils_path = os.path.join(project_root, "utils")
if utils_path not in sys.path:
    sys.path.append(utils_path)

    
import get_data

In [ ]:
# combined_df, patientIds = get_data.get_dataset()
# #SAVE DATAFRAME
# DATA_PATH = get_data.get_dataset_abspath()
# save_path = os.path.join(DATA_PATH, "raw_combined_data.parquet")
# combined_df.to_parquet(save_path, compression="snappy")

Processing training_setA:   0%|          | 0/20336 [00:00<?, ?it/s]

Processing training_setB:   0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
#ACCESS DATAFRAME
DATA_PATH = get_data.get_dataset_abspath()
load_path = os.path.join(DATA_PATH, "raw_combined_data.parquet")
combined_df = pd.read_parquet(load_path)

In [ ]:
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils.impute_data import impute_df_no_nans
import utils.impute_data as impute_clusters
importlib.reload(impute_clusters)

# Choose dataset mode: 1 = full, 2 = first 1000 patients
DATASET_MODE = 1
df_to_impute = (combined_df[combined_df["patient_id"] <= 500].copy() 
                if DATASET_MODE == 2 else combined_df.copy())

# Perform imputation with chosen binning options
df_imputed, replacement_stats = impute_df_no_nans(
    df_to_impute,
    nan_density=0.3,   # <-- You can keep or adjust these parameters

    gender_bins=2,
    age_bins=10,
    hr_bins=5,
    map_bins=5,

)

# Check for any remaining missing values
if df_imputed.isna().sum().sum() == 0:
    print("No NaNs found in the imputed data")

# --- Helper Functions ---

def print_replacement_stats(stats: dict) -> None:
    """Prints formatted replacement statistics."""
    print("\nReplacement statistics (percentages):")
    label_map = {
        "Linear fill": "Linear fill",
        "Cluster fill": "Cluster-mean fill",
        "Nearest cluster fill": "Nearest cluster-mean fill",
        "Initial missing": "Initial missing"
    }
    for col, col_stats in stats.items():
        print(f"Column: {col}")
        for key, value in col_stats.items():
            key_print = label_map.get(key, key.capitalize())
            if isinstance(value, (int, float)):
                print(f"  {key_print}: {value:.2f}%")
            else:
                print(f"  {key_print}: {value}")

def plot_cluster_distribution(df: pd.DataFrame) -> None:
    cluster_counts = df["cluster_id"].value_counts().sort_index()
    cleaned_labels = [
        "_".join(part for part in cid.split("_") if part != "X")
        for cid in cluster_counts.index
    ]
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(cluster_counts)), cluster_counts.values, color="lightblue")
    plt.title("Frequency Distribution of Patient Clusters")
    plt.xlabel("Cluster ID")
    plt.ylabel("Number of Patients")
    plt.tight_layout()
    plt.show()

def plot_cluster_fill(stats: dict) -> None:
    # Filter columns that used the cluster-based method
    cluster_cols = [col for col, stat in stats.items() if stat.get("Method") == "cluster"]


    idx = np.arange(len(cluster_cols))
    cluster_fill = [stats[col]["Cluster fill"] for col in cluster_cols]
    nearest_fill = [stats[col]["Nearest cluster fill"] for col in cluster_cols]

    plt.figure(figsize=(10, 6))
    plt.bar(idx, cluster_fill, label="Cluster-mean fill", color="lightgreen")
    plt.bar(idx, nearest_fill, bottom=cluster_fill, label="Nearest cluster-mean fill", color="lightred")
    plt.xticks(idx, cluster_cols, rotation=90)
    plt.ylabel("Replacement Percentage")
    plt.title("Cluster-Based Mean Replacement Percentages")
    plt.legend()
    plt.tight_layout()
    plt.show()

# --- Execution ---

print_replacement_stats(replacement_stats)
plot_cluster_distribution(df_imputed)
plot_cluster_fill(replacement_stats)








                 HR  O2Sat    Temp    SBP     MAP  DBP  Resp      EtCO2  \
patient_id                                                                
1          0    NaN    NaN  36.778    NaN     NaN  NaN   NaN  32.957657   
           1   97.0   95.0  36.778   98.0  75.330  NaN  19.0  32.957657   
           2   89.0   99.0  36.778  122.0  86.000  NaN  22.0  32.957657   
           3   90.0   95.0  36.778  122.0  88.665  NaN  30.0  32.957657   
           4  103.0   88.5  36.778  122.0  91.330  NaN  24.5  32.957657   

              BaseExcess  HCO3  ...  Platelets    Age  Gender     Unit1  \
patient_id                      ...                                       
1          0   20.714286  46.5  ...      327.5  83.14       0  0.496571   
           1   20.714286  46.5  ...      327.5  83.14       0  0.496571   
           2   20.714286  46.5  ...      327.5  83.14       0  0.496571   
           3   24.000000  46.5  ...      327.5  83.14       0  0.496571   
           4   20.714286